# YOLOv11 Segmentierung testen

In [9]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n-seg.pt")

# Define current screenshot as source
source = "/home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg"

# Run inference on the source
results = model(source)  # list of Results objects

# View results
for r in results:
    print(r.masks)  # print the Masks object containing the detected instance masks#
    break


image 1/1 /home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg: 480x640 8 persons, 6 cars, 2 handbags, 3.9ms
Speed: 1.2ms preprocess, 3.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
      

# Segmentierungsergebnisse visualisieren mit eingebauter `plot()` Funktion

In [13]:
from PIL import Image

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n-seg.pt")

# Run inference on 'bus.jpg'
results = model([
                 "/home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg",
                 "/home/juebrauer/link_to_vcd/10_datasets/01_test_images/testimg003_breakfast_table1.jpg"
                ])  # results list

# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()

    # Save results to disk
    r.save(filename=f"results{i}.jpg")


0: 640x640 8 persons, 6 cars, 3 trucks, 1 traffic light, 2 handbags, 5.8ms
1: 640x640 6 bottles, 2 cups, 1 bowl, 1 sandwich, 1 cake, 1 chair, 3 potted plants, 1 dining table, 5 vases, 5.8ms
Speed: 1.3ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


# Die Segmentierungsergebnisdatenstruktur von YOLOv11 verstehen

In [25]:
from PIL import Image
from ultralytics import YOLO
import numpy as np
import cv2
import os

model = YOLO("yolo11l-seg.pt")

img_paths = [
    "/home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg",
    "/home/juebrauer/link_to_vcd/10_datasets/01_test_images/testimg003_breakfast_table1.jpg"
]
results = model(img_paths)

os.makedirs("first_mask_only", exist_ok=True)

for i, r in enumerate(results):
    im = r.orig_img.copy()  # BGR
    H, W = im.shape[:2]

    if r.masks is not None and len(r.masks.data) > 0:
        # 1) erste Maske holen (Modellgröße)
        mask = r.masks.data[0].cpu().numpy()  # (h,w) float in {0,1}

        # 2) auf Originalgröße bringen (W zuerst!)
        mask_resized = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)

        # 3) Alpha-Blending
        m = mask_resized.astype(bool)
        color = np.array([0, 255, 0], dtype=np.uint8)  # BGR
        alpha = 0.3
        im[m] = ((1 - alpha) * im[m] + alpha * color).astype(np.uint8)

        # 4) Kontur zeichnen (optional)
        cnt_img = (mask_resized * 255).astype(np.uint8)
        cnts, _ = cv2.findContours(cnt_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(im, cnts, -1, (0, 255, 0), 2)

    cv2.imwrite(f"first_mask_only/results{i}.jpg", im)



0: 640x640 6 persons, 7 cars, 6 trucks, 2 handbags, 18.6ms
1: 640x640 9 bottles, 1 wine glass, 3 cups, 1 fork, 1 spoon, 1 bowl, 1 apple, 2 cakes, 3 potted plants, 2 dining tables, 18.6ms
Speed: 1.4ms preprocess, 18.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [15]:
type(m)

numpy.ndarray

In [16]:
m.shape

(640, 640)

In [17]:
type(im)

numpy.ndarray

In [18]:
im.shape

(720, 960, 3)

In [20]:
r.orig_img.shape

(720, 960, 3)

# Segmentierungsmaske anzeigen (Vorschlag von Gemini)

In [57]:
from ultralytics import YOLO
import cv2
import numpy as np

# 1. Load a pre-trained YOLOv11 segmentation model
model = YOLO("yolo11n-seg.pt")

# 2. Run inference on an image
results = model.predict(source="/home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg")

# 3. Extract the first result object
result = results[0]

# 4. Prepare a binary mask and draw contours
# The mask coordinates are in a list of lists format, so we take the first one
# and reshape it to be in the correct format for drawContours

N = len(result)
for object_id in range(N):
    img = result.orig_img.copy()
    b_mask = np.zeros(img.shape[:2], np.uint8)

    contour = result.masks.xy[object_id].astype(np.int32).reshape(-1, 1, 2)
    cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)

    # 5. Isolate the object using the binary mask
    mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR)
    isolated = cv2.bitwise_and(mask3ch, img)

    # 6. Display or save the result
    # cv2.imshow("Isolated Object", isolated)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # Or save the image
    cv2.imwrite(f"isolated_object_{object_id:02}.jpg", isolated)




image 1/1 /home/juebrauer/link_to_vcd/10_datasets/01_test_images/ny_street_scene.jpg: 480x640 8 persons, 6 cars, 2 handbags, 3.6ms
Speed: 1.2ms preprocess, 3.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


In [28]:
len(result)

16

In [32]:
result[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair

In [41]:
result.summary()[0].keys()

dict_keys(['name', 'class', 'confidence', 'box', 'segments'])

In [46]:
len( result.summary()[0]["segments"]["x"] )

156

In [47]:
len( result.summary()[0]["segments"]["y"] )

156

In [48]:
result.masks.xy

[array([[          3,         375],
        [          3,       718.5],
        [      199.5,       718.5],
        [      199.5,       709.5],
        [      193.5,       709.5],
        [        192,         708],
        [        192,       706.5],
        [      190.5,         705],
        [      190.5,       703.5],
        [        189,         702],
        [        189,         699],
        [      187.5,       697.5],
        [      187.5,       694.5],
        [        186,         693],
        [        186,         690],
        [      184.5,       688.5],
        [      184.5,         684],
        [        183,       682.5],
        [        183,       679.5],
        [      181.5,         678],
        [      181.5,         675],
        [        180,       673.5],
        [        180,       670.5],
        [      178.5,         669],
        [      178.5,       667.5],
        [      175.5,       664.5],
        [      175.5,         663],
        [        174,       

In [50]:
result.masks.xy[0].shape

(156, 2)

In [54]:
type(b_mask)

numpy.ndarray

In [55]:
b_mask.min()

np.uint8(0)

In [56]:
b_mask.max()

np.uint8(255)